In [6]:
import graphlab
import re

In [87]:
products = graphlab.SFrame('amazon_baby.gl/')

In [88]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [89]:
products.head(1)

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0


In [90]:
for word in selected_words:
    products[word] = products['review'].apply(lambda x: re.split('\W+', x).count(word))

In [91]:
for word in selected_words:
    print(word, products[word].sum())

('awesome', 3356L)
('great', 50378L)
('fantastic', 1559L)
('amazing', 2295L)
('love', 34810L)
('horrible', 1079L)
('bad', 4732L)
('terrible', 1125L)
('awful', 650L)
('wow', 124L)
('hate', 1143L)


In [92]:
features=selected_words

In [93]:
products = products[products['rating'] != 3]

In [94]:
products['sentiment'] = products['rating'] >=4

In [95]:
train_data,test_data = products.random_split(.8, seed=0)

In [96]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=features,
                                                     validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.335360     | 0.845783          | 0.844433            |

| 5         | 6        | 0.406048     | 0.845783          | 0.844433            |

| 6         | 7        | 0.473729     | 0.845783          | 0.844433            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [97]:
selected_words_model['coefficients']

name,index,class,value,stderr
(intercept),None,1,1.4233821831,0.00871149950679
awesome,None,1,1.02452623986,0.0874098981105
great,None,1,0.75641744842,0.0195559036811
fantastic,None,1,0.790252566113,0.113232664528
amazing,None,1,1.0694527695,0.105101104506
love,None,1,1.2802996333,0.0301205464943
horrible,None,1,-2.15315811591,0.0828632333721
bad,None,1,-0.965482843867,0.0387091323365
terrible,None,1,-2.10350842041,0.0788665823613
awful,None,1,-1.94189641827,0.104507229428


In [98]:
# Out of the 11 words in selected_words, which one got the most positive weight? Which one got the most negative weight? 
# Do these values make sense for you?
selected_words_model['coefficients'].sort('value')

name,index,class,value,stderr
horrible,None,1,-2.15315811591,0.0828632333721
terrible,None,1,-2.10350842041,0.0788665823613
awful,None,1,-1.94189641827,0.104507229428
hate,None,1,-1.29362172077,0.0806965203155
bad,None,1,-0.965482843867,0.0387091323365
wow,None,1,-0.70353862905,0.26403707867
great,None,1,0.75641744842,0.0195559036811
fantastic,None,1,0.790252566113,0.113232664528
awesome,None,1,1.02452623986,0.0874098981105
amazing,None,1,1.0694527695,0.105101104506


In [78]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8444331011289935,
 'auc': 0.6616495767285729,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  101  |
 |      0       |        0        |  248  |
 |      0       |        1        |  5080 |
 |      1       |        1        | 27875 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9149693916068995,
 'log_loss': 0.40649668650993653,
 'precision': 0.8458504020634198,
 'recall': 0.9963897626537032,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   

In [79]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [80]:
diaper_champ_reviews

name,review,rating,awesome,great,fantastic,amazing,love,horrible
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,0,0,0,0,0,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,0,0,0,0,0,0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,0,0,0,0,0,0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,0,0,0,0,1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,0,0,0,1,0,1
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,0,0,0,0,0,0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,0,0,0,0,0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,0,0,0,0,0,0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,0,0,0,0,0,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,0,0,0,0,2,0


In [81]:
predicted_sentiment = selected_words_model.predict(diaper_champ_reviews, output_type='probability')

In [82]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [83]:
diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

name,review,rating,awesome,great,fantastic,amazing,love,horrible
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,0,0,0,0,3,0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,0,2,0,0,1,0
Baby Trend Diaper Champ,I have been using thisdiaper pail for 41/2 ...,5.0,0,2,0,0,1,0
Baby Trend Diaper Champ,I love this diaper pail!It's so easy to use a ...,5.0,0,0,0,0,2,0
Baby Trend Diaper Champ,I've worked with kidsmore than half my life. ...,5.0,0,0,0,0,2,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,0,0,0,0,2,0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,0,0,0,0,2,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,0,0,0,0,2,0
Baby Trend Diaper Champ,I have a two-year-old sonand I love the Diaper ...,5.0,0,0,0,0,2,0
Baby Trend Diaper Champ,I love this diaper pail.It keeps the diapers ...,4.0,0,0,0,0,2,0


In [85]:
target = 'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'

In [86]:
diaper_champ_reviews[diaper_champ_reviews['review'] == a]

Columns:
	name	str
	review	str
	rating	float
	awesome	int
	great	int
	fantastic	int
	amazing	int
	love	int
	horrible	int
	bad	int
	terrible	int
	awful	int
	wow	int
	hate	int
	sentiment	int
	predicted_sentiment	float

Rows: Unknown

Data:
+-------------------------+-------------------------------+--------+---------+-------+
|           name          |             review            | rating | awesome | great |
+-------------------------+-------------------------------+--------+---------+-------+
| Baby Trend Diaper Champ | Baby Luke can turn a clean... |  5.0   |    0    |   0   |
+-------------------------+-------------------------------+--------+---------+-------+
+-----------+---------+------+----------+-----+----------+-------+-----+------+
| fantastic | amazing | love | horrible | bad | terrible | awful | wow | hate |
+-----------+---------+------+----------+-----+----------+-------+-----+------+
|     0     |    0    |  0   |    0     |  0  |    0     |   0   |  0  |  0   |
+-----------+---------+------+----------+-----+----------+-------+-----+------+
+-----------+---------------------+
| sentiment | predicted_sentiment |
+-----------+---------------------+
|     1     |    0.722616517384   |
+-----------+---------------------+
[? rows x 16 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.